In [2]:
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.models import Sequential
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
import re
import numpy as np
import pandas as pd
from pprint import pprint
import glob


Using TensorFlow backend.


In [3]:
activationtype="tanh"

In [4]:
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path

mallet_model=gensim.models.LdaModel.load('./models/model-050')

pprint(mallet_model.print_topics(30))

model=gensim.models.wrappers.ldamallet.malletmodel2ldamodel(mallet_model, gamma_threshold=0.001, iterations=50)

[(49,
  '0.047*"live" + 0.042*"city" + 0.027*"place" + 0.025*"move" + 0.025*"home" + '
  '0.023*"area" + 0.021*"house" + 0.019*"build" + 0.017*"new_york" + '
  '0.016*"local"'),
 (5,
  '0.023*"religious" + 0.023*"religion" + 0.018*"freedom" + 0.017*"christian" '
  '+ 0.016*"church" + 0.016*"god" + 0.015*"belief" + 0.014*"society" + '
  '0.013*"moral" + 0.013*"world"'),
 (44,
  '0.221*"year" + 0.102*"time" + 0.084*"long" + 0.043*"ago" + 0.029*"start" + '
  '0.023*"term" + 0.022*"back" + 0.020*"end" + 0.018*"begin" + 0.017*"happen"'),
 (12,
  '0.061*"thing" + 0.044*"talk" + 0.038*"guy" + 0.035*"happen" + 0.026*"stop" '
  '+ 0.019*"bad" + 0.018*"real" + 0.018*"start" + 0.017*"hear" + 0.017*"head"'),
 (20,
  '0.107*"child" + 0.076*"family" + 0.045*"parent" + 0.041*"kid" + '
  '0.031*"friend" + 0.029*"life" + 0.028*"young" + 0.025*"age" + 0.023*"live" '
  '+ 0.023*"mother"'),
 (2,
  '0.067*"gun" + 0.031*"people" + 0.028*"control" + 0.027*"kill" + '
  '0.026*"fear" + 0.022*"act" + 0.021*"vio

In [ ]:
##vaxx files
files = glob.glob('../documents/vaxx/*.txt')
print(len(files))
files=np.random.choice(files,int(len(files)))
print(len(files))
data = []
for f in files:
    with open(f, 'r') as fin:
        data.append(fin.read())

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))

In [81]:
files = glob.glob('../documents/comments/comments/*.txt')
print(len(files))
files=np.random.choice(files,int(len(files)/200))
print(len(files))
data = []
for f in files:
    with open(f, 'r') as fin:
        data.append(fin.read())

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))

445441
2227


In [82]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.max_length = 9297925
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

['if', 'he', 'is', 'that', 'reckless', 'with', 'his', 'own', 'money', 'how', 'much', 'more', 'reckless', 'will', 'be', 'be', 'with', 'the', 'nation', 'money', 'when', 'there', 'is', 'so', 'much', 'more', 'to', 'spend', 'and', 'he', 'is', 'not', 'responsible', 'for', 'paying', 'the', 'debts', 'he', 'incurs', 'with', 'it']
[['reckless', 'money', 'much', 'reckless', 'nation', 'money', 'much', 'spend', 'responsible', 'pay', 'debt', 'incur']]


In [83]:
# Create Dictionary
id2word = mallet_model.id2word

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

X=np.asarray(model.inference(corpus)[0])
num_topics=len(X[0])
##normalize it
X=(X.T/np.sum(X,axis=1)).T
Y=np.asarray([len(text) for text in texts])

[[(10, 2), (424, 2), (429, 1), (754, 1), (766, 1), (1096, 1), (1177, 1), (1797, 1), (15174, 2)]]


In [84]:
##print((X[idx]))
topics4X=np.argmax(X[1:80],axis=1)
print(topics4X)
idx=1
topic4X=np.argmax(X[idx])
print(" ".join(data_words[idx]))
print(model.show_topic(topic4X))
##minor problem: the token br is not removed

[15 22 37 46 24 49 13 11 35 36  7 24 17 45 32 42 25 23  6 10 36 26 10 19
 13 27 41 49  1 47 49 11 42  4 47  3  0 28 29  4 20 16 48 23 33 25 16 49
 41  0  8  7  0  7 26 17 45 12 10 13  2 26 15 25 45 31 46 10 48 16 13 10
 39 39 27 16 10 23  3]
let see financial institution that promotes export of us goods that helps us compete globally against industrial countries that give their citizens access to way more trade support than that provided by ex im that plays critical part in sustaining thousands of small companies and hundreds of thousands of jobs and that has increased the treasury to the tune of billion over its history obviously this is bad for america and has to be killed not that surprised the right wing extremists are doing this it is their brand of political theater although of course have not heard their plan to replace ex im commercial banks won fill this void but the most distressing part of this to life long republican like me is the inability of our party leadership to stand

In [10]:
##don't randomly run this. this will overwrite the mode. If you accidentally ran this, run the load function 3 blocks below
normalized_model=Sequential()
normalized_dense_layer=Dense(100,input_shape=[num_topics,],activation=activationtype)
normalized_dense_layer_2=Dense(1)
normalized_model.add(normalized_dense_layer)
normalized_model.add(normalized_dense_layer_2)
normalized_model.compile(loss="mean_squared_error", optimizer='adam', metrics=['accuracy'])
normalized_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 5,201
Trainable params: 5,201
Non-trainable params: 0
_________________________________________________________________


In [77]:
normalized_model.fit(X, Y, epochs=10000, batch_size=20000)

Epoch 1/10000
22272/22272 [==============================] - 0s 9us/step - loss: 112.7849 - acc: 0.0638
Epoch 2/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.8028 - acc: 0.0627
Epoch 3/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.7796 - acc: 0.0638
Epoch 4/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.7752 - acc: 0.0668
Epoch 5/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.8413 - acc: 0.0688
Epoch 6/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.8152 - acc: 0.0672
Epoch 7/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.7617 - acc: 0.0645
Epoch 8/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.7770 - acc: 0.0624
Epoch 9/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.7513 - acc: 0.0639
Epoch 10/10000
22272/22272 [==============================] - 0s

22272/22272 [==============================] - 0s 2us/step - loss: 112.5160 - acc: 0.0643
Epoch 80/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.5144 - acc: 0.0647
Epoch 81/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.5077 - acc: 0.0643
Epoch 82/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.5084 - acc: 0.0641
Epoch 83/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.5048 - acc: 0.0646
Epoch 84/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.4972 - acc: 0.0641
Epoch 85/10000
22272/22272 [==============================] - 0s 4us/step - loss: 112.5131 - acc: 0.0628
Epoch 86/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.5511 - acc: 0.0607
Epoch 87/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.5268 - acc: 0.0618
Epoch 88/10000
22272/22272 [==============================] - 0s 2us/s

22272/22272 [==============================] - 0s 2us/step - loss: 112.0261 - acc: 0.0656
Epoch 234/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.0585 - acc: 0.0678
Epoch 235/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.0428 - acc: 0.0665
Epoch 236/10000
22272/22272 [==============================] - 0s 3us/step - loss: 112.0193 - acc: 0.0661
Epoch 237/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.0156 - acc: 0.0662
Epoch 238/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.0011 - acc: 0.0657
Epoch 239/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.9940 - acc: 0.0654
Epoch 240/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.9916 - acc: 0.0653
Epoch 241/10000
22272/22272 [==============================] - 0s 2us/step - loss: 112.0086 - acc: 0.0622
Epoch 242/10000
22272/22272 [==============================] -

22272/22272 [==============================] - 0s 2us/step - loss: 111.5412 - acc: 0.0661
Epoch 388/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5300 - acc: 0.0664
Epoch 389/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5208 - acc: 0.0654
Epoch 390/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5411 - acc: 0.0621
Epoch 391/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5277 - acc: 0.0634
Epoch 392/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5105 - acc: 0.0660
Epoch 393/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5133 - acc: 0.0663
Epoch 394/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5262 - acc: 0.0669
Epoch 395/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.5113 - acc: 0.0665
Epoch 396/10000
22272/22272 [==============================] -

22272/22272 [==============================] - 0s 2us/step - loss: 111.0298 - acc: 0.0655
Epoch 542/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.0301 - acc: 0.0646
Epoch 543/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.0357 - acc: 0.0637
Epoch 544/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.0359 - acc: 0.0639
Epoch 545/10000
22272/22272 [==============================] - 0s 3us/step - loss: 111.0198 - acc: 0.0652
Epoch 546/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.0176 - acc: 0.0661
Epoch 547/10000
22272/22272 [==============================] - 0s 2us/step - loss: 111.0087 - acc: 0.0659
Epoch 548/10000
22272/22272 [==============================] - 0s 3us/step - loss: 111.0380 - acc: 0.0639
Epoch 549/10000
22272/22272 [==============================] - 0s 3us/step - loss: 111.0326 - acc: 0.0638
Epoch 550/10000
22272/22272 [==============================] -

...

22272/22272 [==============================] - 0s 2us/step - loss: 88.2635 - acc: 0.0696
Epoch 9353/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1859 - acc: 0.0727
Epoch 9354/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1604 - acc: 0.0755
Epoch 9355/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1587 - acc: 0.0752
Epoch 9356/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1523 - acc: 0.0751
Epoch 9357/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1641 - acc: 0.0737
Epoch 9358/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1481 - acc: 0.0752
Epoch 9359/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1770 - acc: 0.0789
Epoch 9360/10000
22272/22272 [==============================] - 0s 2us/step - loss: 88.1565 - acc: 0.0777
Epoch 9361/10000
22272/22272 [==============================] -

22272/22272 [==============================] - 0s 2us/step - loss: 87.8193 - acc: 0.0740
Epoch 9507/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8184 - acc: 0.0753
Epoch 9508/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8367 - acc: 0.0761
Epoch 9509/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8312 - acc: 0.0762
Epoch 9510/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8073 - acc: 0.0733
Epoch 9511/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8201 - acc: 0.0718
Epoch 9512/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8088 - acc: 0.0726
Epoch 9513/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8111 - acc: 0.0744
Epoch 9514/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.8022 - acc: 0.0744
Epoch 9515/10000
22272/22272 [==============================] -

22272/22272 [==============================] - 0s 2us/step - loss: 87.5063 - acc: 0.0757
Epoch 9661/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.5020 - acc: 0.0757
Epoch 9662/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.4825 - acc: 0.0729
Epoch 9663/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.4915 - acc: 0.0718
Epoch 9664/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.5030 - acc: 0.0717
Epoch 9665/10000
22272/22272 [==============================] - 0s 3us/step - loss: 87.4848 - acc: 0.0720
Epoch 9666/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.4820 - acc: 0.0735
Epoch 9667/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.4858 - acc: 0.0746
Epoch 9668/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.4832 - acc: 0.0740
Epoch 9669/10000
22272/22272 [==============================] -

22272/22272 [==============================] - 0s 2us/step - loss: 87.2236 - acc: 0.0709
Epoch 9815/10000
22272/22272 [==============================] - 0s 3us/step - loss: 87.2008 - acc: 0.0709
Epoch 9816/10000
22272/22272 [==============================] - 0s 3us/step - loss: 87.1625 - acc: 0.0742
Epoch 9817/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.3194 - acc: 0.0798
Epoch 9818/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.2685 - acc: 0.0782
Epoch 9819/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.1512 - acc: 0.0726
Epoch 9820/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.3411 - acc: 0.0683
Epoch 9821/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.3415 - acc: 0.0689
Epoch 9822/10000
22272/22272 [==============================] - 0s 2us/step - loss: 87.1532 - acc: 0.0722
Epoch 9823/10000
22272/22272 [==============================] -

22272/22272 [==============================] - 0s 3us/step - loss: 86.8655 - acc: 0.0704
Epoch 9969/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8290 - acc: 0.0724
Epoch 9970/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8388 - acc: 0.0746
Epoch 9971/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8508 - acc: 0.0755
Epoch 9972/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8290 - acc: 0.0741
Epoch 9973/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8149 - acc: 0.0731
Epoch 9974/10000
22272/22272 [==============================] - 0s 2us/step - loss: 86.8125 - acc: 0.0731
Epoch 9975/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8114 - acc: 0.0732
Epoch 9976/10000
22272/22272 [==============================] - 0s 3us/step - loss: 86.8110 - acc: 0.0731
Epoch 9977/10000
22272/22272 [==============================] -

In [46]:
print(normalized_dense_layer.input_shape)
print(normalized_dense_layer.output_shape)
print(normalized_dense_layer_2.input_shape)
print(normalized_dense_layer_2.output_shape)
print(X.shape)
print(Y.shape)
normalized_model.summary()
weights=normalized_model.get_weights()
weights=normalized_model.get_weights()
print(abs(weights[0]).mean())
print(abs(weights[1]).mean())

(None, 50)
(None, 100)
(None, 100)
(None, 1)
(890, 50)
(890,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 5,201
Trainable params: 5,201
Non-trainable params: 0
_________________________________________________________________
2.4518983
0.5618683


In [78]:
normalized_model.save("./models/"+activationtype+"-"+str(num_topics))

In [5]:
import keras.models
normalized_model=keras.models.load_model("./models/"+activationtype+"-"+str(num_topics))
normalized_model.summary()
print(normalized_model.evaluate(X,Y))

NameError: name 'num_topics' is not defined

In [85]:
##predict and evaluate pearson r
normalized_prediction=normalized_model.predict(X).squeeze()
print(str(len(normalized_prediction))+" examples")
print("predicted length")
print(normalized_prediction[0:10])
print("actual length")
print(Y[0:10])
from scipy.stats.stats import pearsonr
print(pearsonr(normalized_prediction,Y))

2227 examples
predicted length
[ 14.3315525  57.4492     24.12053    38.591724   12.248506  117.02874
  50.607845   12.222472   55.15383    48.453884 ]
actual length
[ 12  69  28  35  11 126  73  10  49  49]
(0.9405711728897699, 0.0)


In [91]:
#randomly sample to see the difference
normalized_prediction=normalized_model.predict(X).squeeze()
randX=abs(np.random.randn(len(Y),num_topics))
randX=(randX.T/randX.sum(axis=1)).T
randpred=normalized_model.predict(randX).squeeze()
print("random prediction mean: "+str(randpred.mean()))
print("random prediction std: "+str(np.std(randpred)))
print("pearson of random: "+str(pearsonr(randpred,Y)))
print("pearson of actual prediction: "+str(pearsonr(normalized_prediction,Y)))
print("Y mean: "+str(Y.mean()))
pred=normalized_model.predict(X)
print("Actual Prediction mean: "+str(pred.mean()))
print("Y std: "+str(np.std(Y)))
print("Actual Prediction std: "+str(np.std(pred)))

random prediction mean: 61.71489
random prediction std: 26.726973
pearson of random: (0.009911380179382385, 0.6401580556576757)
pearson of actual prediction: (0.9405711728897699, 0.0)
Y mean: 37.72878311629996
Actual Prediction mean: 37.6388
Y std: 33.0545411021382
Actual Prediction std: 30.412462
